In [ ]:
#preamble to analysis
import pandas as pd
import numpy as np
from IPython.display import Markdown as md
from IPython.display import HTML, display
from time import sleep

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True,connected=True)
from itables import show as iShow
import itables.options as opt
opt.classes="display compact cell-border"
opt.maxBytes = 0
opt.column_filters = "footer"
opt.dom="lrtip"

opt.order = []  # no sorting

In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup
import openFF.common.file_handlers as fh 
import openFF.common.text_handlers as th
import openFF.common.handles as hndl

In [ ]:
def make_link(row):
    return th.wrap_URL_in_html(row.link_fn,row.api10)

In [ ]:
df = fh.get_df(hndl.curr_data,cols=['DisclosureId','api10','OperatorName','ingKeyPresent','bgStateName','bgCountyName',
                        'TotalBaseWaterVolume','WellName','date'])
# c1 = df.bgStateName=='pennsylvania'
# c2 = df.bgCountyName.isin(['jefferson'])
gb1 = df.groupby('DisclosureId',as_index=False)[['api10','date','OperatorName','bgStateName','bgCountyName','WellName',
                        'TotalBaseWaterVolume']].first()
gb2 = df.groupby('DisclosureId',as_index=False)['ingKeyPresent'].any()
mg = pd.merge(gb1,gb2,on='DisclosureId',how='left')
mg['apicode'] = mg.api10.str[:5]
mg['link_fn'] = f'disclosures/'+mg.apicode+'/'+mg.DisclosureId+'.html'
mg['API'] = mg.apply(make_link,axis=1)
mg['haschem'] = np.where(mg.ingKeyPresent,'YES','NO')

mg = mg[['API','date','OperatorName','bgStateName','bgCountyName',
          'WellName','TotalBaseWaterVolume','haschem']].reset_index(drop=True)
mg = mg.rename({'API':'API Number and link', 'OperatorName':'Operator','bgStateName':'State',
               'bgCountyName':'County','WellName':'Well Name','TotalBaseWaterVolume':'Water (gal.)',
               'haschem':'has chemical records'}, axis=1)
# mg.columns

In [ ]:
from itables import JavascriptCode
iShow(mg,columnDefs=[{"width": "80px", "targets": 2},
                     {"render": JavascriptCode("$.fn.dataTable.render.number(',', '.', 0)"),"targets": 6}],
      classes="display compact cell-border",
      style="table-layout:auto;width:90%;float:center")

<script>
    document.querySelectorAll('a.toggle-vis').forEach((el) => {
    el.addEventListener('click', function (e) {
        e.preventDefault();
 
        let columnIdx = e.target.getAttribute('data-column');
        let column = table.column(columnIdx);
 
        // Toggle the visibility
        column.visible(!column.visible());
    });
});
</script>